In [1]:
import math, copy, sys

import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F
import import_ipynb
from Elements import *

importing Jupyter notebook from Elements.ipynb
importing Jupyter notebook from MoveData.ipynb


## The Encoder 

All the subcomponents of the `EncoderLayer()` are thoroughly explained in Elements.ipynb

the inputs to the encoder are sequences of integers shaped (batch size, sequence length)

the output are sequences of vectors shaped (batch size, sequence length, embedding dimensions)

<img src="../saved/images/encoderchart.png">

The diagram above mirrors the code below. As you follow the diagram upwards, you can see the data pass through the same modules as you move down the lines of code. `x` in our code is the sequence of tokens, the black line, moving through the diagram.

The box labeled Nx is the `EncoderLayer` function below, that is repeated an arbitrary Nx number of times (6 in the paper). 

In [2]:
class EncoderLayer(nn.Module):
    def __init__(self, emb_dim, heads, dropout=0.1):
        super().__init__()
        self.norm_1 = Norm(emb_dim)
        self.dropout_1 = nn.Dropout(dropout)
        self.attn = MultiHeadAttention(heads, emb_dim, dropout=dropout)
        self.norm_2 = Norm(emb_dim)
        self.ff = FeedForward(emb_dim, dropout=dropout)
        self.dropout_2 = nn.Dropout(dropout)
        
    def forward(self, x, mask):
        x2 = self.norm_1(x)
        x2_attn, x2_scores = self.attn(x2,x2,x2,mask)
        x = x + self.dropout_1(x2_attn)
        x2 = self.norm_2(x)
        x = x + self.dropout_2(self.ff(x2))
        return x

In the forward function of `Encoder` you can see that just as in the diagram, the embedding occurs first `x = self.embed(source_sequence)` followed by positional encoding `x = self.pe(x)`. Then the `EncoderLayer(emb_dim, heads, dropout)` is repeated an `n_layers` number of times. 

Within the gray box in the diagram you can see the first step is the Multi-Head Attention. In our implementation we do a normalization first. But the the data splits into 3 arrows before going into Multi-Head Attention, just like we use the `x2` three times in `x2_attn, x2_scores = self.attn(x2,x2,x2,mask)`. This signifies that we used the same sequence of vectors for the generation of `num_heads` number of query `q`, key `k` and  value `v` vectors. 

Another minor difference is the use of dropout. Dropout is the random zeroing out of certain neurons or activations. The line above that reads `x = x + self.dropout_1(x2_attn)` could be written `x = x + x2_attn` to make it resemble the residual connection in the diagram. The line in the diagram that branches and goes around the Multi-Head Attention layer and into the "Add & Norm" module, is the residual. In english, it means that the input to the norm layer is the sum of x after it passes through the Multi-Head Attention with a copy of x that has not passed through the Multi-Head Attention.

In the same way, the output of the feed forward layer `self.ff(x2)` is added to itself before normalized again. This is the 2nd residual in the diagram. 

In [ ]:
class Encoder(nn.Module):
    def __init__(self, vocab_size, emb_dim, n_layers, heads, dropout):
        super().__init__()
        self.n_layers = n_layers
        self.embed = Embedder(vocab_size, emb_dim)
        self.pe = PositionalEncoder(emb_dim, dropout=dropout)
        self.layers = get_clones(EncoderLayer(emb_dim, heads, dropout), n_layers)
        self.norm = Norm(emb_dim)
    def forward(self, source_sequence, source_mask):
        '''
        input:
            source_sequence (sequence of source tokens) of shape (batch size, sequence length)
            source_mask (mask over input sequence) of shape (batch size, 1, sequence length)
        output: sequence of vectors after embedding, postional encoding, attention and normalization
            shape (batch size, sequence length, embedding dimensions)
        '''
        x = self.embed(source_sequence)
        x = self.pe(x)
        for i in range(self.n_layers):
            x = self.layers[i](x, source_mask)
        x = self.norm(x)
        return x